In [180]:
import sys
sys.path.append('..')
import cmath
import numpy as np
from scipy.sparse import diags
from scipy.linalg import eigh
import matplotlib.pyplot as plt

In [169]:
def update_couplings(matrix, indices, value):
    """Helper function to update symmetric off-diagonal matrix elements."""
    for i, j in indices:
        matrix[i, j] = value
        matrix[j, i] = value  # Ensure symmetry

def C_matrix(N, Cs, Cjj, C0, C0big, Cin):
    # Create the diagonal, sub-diagonal, and super-diagonal values
    diagonals = [-Cjj * np.ones(N), (2 * Cjj + C0) * np.ones(N + 1), -Cjj * np.ones(N)]
    matrix = diags(diagonals, offsets=[-1, 0, 1]).toarray()

    # Update specific values at the edges
    matrix[0, 0] = Cin + Cs
    matrix[1, 1] = C0big + Cs + Cjj
    matrix[-2, -2] = C0big + Cs + Cjj
    matrix[-1, -1] = Cs

    # Apply couplings using the helper function for symmetry
    update_couplings(matrix, [(1, 0), (-2, -1)], -Cs)

    return matrix

def L_inv_matrix(N, Ljj, Ls):
    # Create the diagonal, sub-diagonal, and super-diagonal values
    diagonals = [(-1 / Ljj) * np.ones(N), (2 / Ljj) * np.ones(N + 1), (-1 / Ljj) * np.ones(N)]
    matrix = diags(diagonals, offsets=[-1, 0, 1]).toarray()

    # Update specific values at the edges
    matrix[0, 0] = 1 / Ls
    matrix[-1, -1] = 1 / Ls
    matrix[1, 1] = 1 / Ls + 1 / Ljj
    matrix[-2, -2] = 1 / Ls + 1 / Ljj

    # Apply couplings using the helper function for symmetry
    update_couplings(matrix, [(1, 0), (-2, -1)], -1 / Ls)

    return matrix

def jja_eigensys(Cs,Ls,Cjj,Ljj,C0, C0big, Cin, N):

    eigenvalues_C, eigenvectors_C = eigh(C_matrix(N,Cs,Cjj,C0,C0big,Cin))
    Lambda_inv_sqrt = np.diag(1 / np.sqrt(eigenvalues_C))
    C_inv_sqrt = np.dot(eigenvectors_C, np.dot(Lambda_inv_sqrt, eigenvectors_C.T)) # spectral decomposition of C^-1/2
    matrix_operation = np.dot(C_inv_sqrt, np.dot(L_inv_matrix(N,Ljj, Ls), C_inv_sqrt)) # C^-1/2 L^-1 C^-1/2
    eigvals, eigvecs = eigh(matrix_operation) # eigenvalues and eigvecs of C^-1/2 L^-1 C^-1/2
    return np.sqrt(eigvals), eigvecs # Returns \omega_k, \Phi_k

In [181]:
### 500 nm
Cs=8.911e-15
Ls = 102.48e-9

Cjj = 29e-15
Ljj = 3e-9
C0 = 5.7e-15
C0big = C0*120
Cin = 1e-15
N=7

cmatrix = C_matrix(N,Cs,Cjj,C0,C0big,Cin)
linvmatrix = L_inv_matrix(N, Ljj, Ls)
evals,eigvecs = jja_eigensys(Cs,Ls,Cjj,Ljj,C0,C0big, Cin, N)
evals/2/np.pi*1e-9

array([9.60219178e-08, 2.19906616e+00, 4.99432727e+00, 5.26668509e+00,
       1.39015189e+01, 1.59687050e+01, 1.64571362e+01, 1.66178606e+01])

In [ ]:
C0big_array = np.geomspace(C0,C0*500,50)
f0_array = np.zeros_like(C0big_array)

for i, C0big in enumerate(C0big_array):
    cmatrix = C_matrix(N,Cs,Cjj,C0,C0big,Cin)
    linvmatrix = L_inv_matrix(N, Ljj, Ls)
    evals,eigvecs = jja_eigensys(Cs,Ls,Cjj,Ljj,C0,C0big, Cin, N)
    evals/2/np.pi*1e-9

In [ ]:

fig, ax = plt.subplots(1,1)
ax.plot()

In [179]:
### 150 nm
Cs=10.31e-15
Ls = 131.84e-9

Cjj = 29e-15
Ljj = 3e-9
C0 = 0.057e-15
C0big = C0*120
Cin = 1e-15
N=7

cmatrix = C_matrix(7,Cs,Cjj,C0,C0big,Cin)
linvmatrix = L_inv_matrix(N, Ljj, Ls)
evals,eigvecs = jja_eigensys(Cs,Ls,Cjj,Ljj,C0, C0big,Cin, N)
evals/2/np.pi*1e-9

C:\Users\jc269556\AppData\Local\Temp\ipykernel_10424\2615845286.py:46: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals), eigvecs # Returns \omega_k, \Phi_k


array([        nan,  4.1334236 ,  4.31685242, 13.37252896, 17.02010531,
       17.05111567, 17.05680467, 17.05856876])

In [39]:
evals/2/np.pi*1e-9

array([1.24117805e-06, 5.26993878e+00, 5.26993878e+00, 1.64099080e+01,
       1.64538225e+01, 1.64618977e+01, 1.64645921e+01, 1.64656494e+01])